In [1]:
import os
import pandas as pd
from pathlib import Path

In [2]:
home = '/home/sshankar'
ibc_folder = 'ibc'

In [3]:
# The sessions.csv file contains information on which task 
# was acquired in which session
# Use it to also get the list of subjects and sessions

sessfile = os.path.join(home, ibc_folder, 'public_analysis_code/ibc_data/sessions.csv')
sess_df = pd.read_csv(sessfile)
subjects = sess_df.subject
sessions = sess_df.columns[1:]

In [4]:
# What is the task of interest?
task = 'hcp1'

In [5]:
subjects

0     sub-01
1     sub-02
2     sub-04
3     sub-05
4     sub-06
5     sub-07
6     sub-08
7     sub-09
8     sub-11
9     sub-12
10    sub-13
11    sub-14
12    sub-15
Name: subject, dtype: object

In [5]:
# For each subject, find which sessions contain the task of interest
task_sess = []

for i in range(len(subjects)):
#     if subjects[i] in subs:
    ser = sess_df.iloc[i,:]
    ids = ser.str.contains(task)==True
    sess = ser.loc[ids].keys().tolist()
    task_sess.append(sess)

In [ ]:
task_sess

In [ ]:
# Some subjects have an extra session. If so, ignore the first session.
# Number of sessions per task
if task=='raiders':
    nsess = 2
elif task=='clips':
    nsess = 3

new_tsess = []
for s in range(len(task_sess)):
    if len(task_sess[s]) > nsess:
        l = len(task_sess[s])
        new_tsess.append(task_sess[s][l-nsess:])
    else:
        new_tsess.append(task_sess[s])

In [6]:
new_tsess = task_sess

In [9]:
# Some useful directories
data_dir_local = os.path.join(home, task, 'sourcedata')
data_dir_drago = 'sshankar@drago:/storage/store/data/ibc/derivatives/'
events_dir_drago = 'sshankar@drago:/storage/store/data/ibc/sourcedata/'
fmap_dir_drago = 'sshankar@drago:/storage/store/data/ibc/sourcedata/'

In [ ]:
# Create directories for subjects and sessions
for i, sub in enumerate(subjects):
    for ses in new_tsess[i]:
        ses_path = os.path.join(data_dir_local, sub, ses)
        if not os.path.isdir(ses_path):
            os.makedirs(ses_path)

In [ ]:
# To copy data files
for i, sub in enumerate(subjects):
    for ses in new_tsess[i]:
        ses_path = os.path.join(data_dir_local, sub, ses)
        os.chdir(ses_path)
        remote_data_file = os.path.join(data_dir_drago, sub, ses, 'func/dc*')
#             remote_data_file = os.path.join(data_dir_drago, sub, ses, 'anat/*nii.gz')
        os.system('scp %s .' % (remote_data_file) )

In [9]:
# To copy events files
for i, sub in enumerate(subjects):
    for ses in new_tsess[i]:
        ses_path = os.path.join(data_dir_local, sub, ses)
        os.chdir(ses_path)
        remote_data_file = os.path.join(events_dir_drago, sub, ses, 'func/*tsv')
        os.system('scp %s .' % (remote_data_file) )

In [10]:
# To copy fmap files
for i, sub in enumerate(subjects):
    if i < 5:
        for ses in new_tsess[i]:
            ses_path = os.path.join(data_dir_local, sub, ses, 'fmap')
            os.chdir(ses_path)
            remote_data_file = os.path.join(events_dir_drago, sub, ses, 'fmap/*dir*epi*gz')
            os.system('scp %s .' % (remote_data_file) )

In [ ]:
# To copy movement parameter files
# for i, sub in enumerate(subjects):
i=12
sub = subjects[i]
for ses in new_tsess[i]:
    ses_path = os.path.join(data_dir_local, sub, ses)
    os.chdir(ses_path)
    remote_data_file = os.path.join(data_dir_drago, sub, ses, 'func/rp*txt')
    os.system('scp %s .' % (remote_data_file) )

In [ ]:
# To move from one local directory to another
data_dir_from = os.path.join(home, task, 'derivatives')
data_dir_to = os.path.join(home, task, '3mm')
for i, sub in enumerate(subjects):
    for ses in task_sess[i]:
        ses_path = os.path.join(data_dir_to, sub, ses)
        if os.path.isdir(ses_path):
            os.chdir(ses_path)
            remote_data_file = os.path.join(data_dir_from, sub, ses, 'rp*txt')
            os.system('mv %s .' % (remote_data_file) )